In [2]:
%load_ext autoreload
%autoreload 2
import torch
import yaml
import logging
from imp import reload
import matplotlib.pyplot as plt
import yaml
import numpy as np
import os
from tqdm import tqdm
from load import load_data
tqdm.get_lock().locks = []
reload(logging)
logger = logging.getLogger(__name__).setLevel(logging.INFO)

# Load data

In [3]:
# declare data parameters
# out 00 -> without bln, 0.004
par_train = {'data_dir'         : '/data/LiftPose3D/capture', # change the path 
             'out_dir'          : '/data/LiftPose3D/capture/out_across_animals',
             'train_session_id' : [3],
             'test_session_id'  : [0],
             'test_cam_id'      : [0]}

# merge with training parameters
par_data = yaml.full_load(open('param.yaml', "rb"))
par = {**par_data["data"], **par_train}

In [4]:
train_3d, train_2d, train_keypoints, test_3d, test_2d, test_keypoints, cams, train_fid, test_fid, train_2d_sh, test_2d_sh  = load_data(par_train, return_frame_info=True)

Loading data...
OK


# Train

In [5]:
from liftpose.main import train as lp3d_train
from liftpose.lifter.augmentation import random_project, perturb_pose, project_to_cam
from liftpose.vision_3d import intrinsic_matrix, calculate_bone_length
from liftpose.preprocess import obtain_projected_stats


angle_aug = {'eangles': {0: [[-5,5], [-5, 5], [-5,5]]},
            'axsorder': 'zyx',
            'vis': None,
            'tvec': None,
            'intr': None}

bone_len = calculate_bone_length(test_3d, par_data['vis']['bones'])
avg_bone_len= np.nanmean(bone_len, axis=0)
std_bone_len= np.nanstd(bone_len, axis=0)

pose_aug = {'perturb': 2,
            'child': par_data["vis"]["child"],
            'bones': par_data['vis']['bones'],
            'avg_bone_len': avg_bone_len,
            'std_bone_len': std_bone_len}


stats = obtain_projected_stats({'train_3d':train_3d},
                               angle_aug['eangles'], 
                               angle_aug['axsorder'], 
                               angle_aug['vis'],
                               angle_aug['tvec'],
                               angle_aug['intr'],
                               par['roots'], 
                               par['target_sets'],
                               par['out_dir'],
                               load_existing=False,
                               th=1)

#aug = [perturb_pose(**pose_aug), random_project(**angle_aug)]
aug = [random_project(**angle_aug)]

lp3d_train(train_2d=None, test_2d={'test':test_2d},
           train_3d={'train':train_3d}, test_3d={'test':test_3d},
           train_keypts={'train':train_keypoints}, test_keypts={'test':test_keypoints}, 
           roots=par['roots'],
           target_sets=par['target_sets'],
           out_dir=par['out_dir'], 
           training_kwargs={"epochs":30},
                            #"resume":True,
                            #"load":par['out_dir'] + '/ckpt_last.pth.tar'},
           augmentation=aug,
           stats=stats,
           norm_2d=True
          )

[preprocess.py:447]:INFO:Bootstrapping mean and variance...
[preprocess.py:534]:INFO:Expected error for obtaining projection stats: 412.86111285717277


/home/gosztolai/Dropbox/github/LiftPose3D/liftpose/preprocess.py:201: RuntimeWarning: invalid value encountered in true_divide
  tmp /= np.linalg.norm(tmp, ord='fro', axis=(1,2), keepdims=True)


[preprocess.py:534]:INFO:Expected error for obtaining projection stats: 0.11413355829789695


/home/gosztolai/Dropbox/github/LiftPose3D/liftpose/preprocess.py:201: RuntimeWarning: invalid value encountered in true_divide
  tmp /= np.linalg.norm(tmp, ord='fro', axis=(1,2), keepdims=True)
/home/gosztolai/Dropbox/github/LiftPose3D/liftpose/preprocess.py:201: RuntimeWarning: invalid value encountered in true_divide
  tmp /= np.linalg.norm(tmp, ord='fro', axis=(1,2), keepdims=True)


[main.py:252]:INFO:Saving pre-processed 2D data at /data/LiftPose3D/capture/out_across_animals/stat_2d.pth.tar.
[main.py:271]:INFO:Saving pre-processed 3D data at /data/LiftPose3D/capture/out_across_animals/stat_3d.pth.tar.
[main.py:299]:INFO:Starting training model.
[main.py:309]:DEBUG:
==================Options=================
[main.py:310]:DEBUG:{   'batch_size': 64,
    'data_dir': '/data/LiftPose3D/capture/out_across_animals',
    'drop_input': 0.0,
    'dropout': 0.5,
    'epochs': 30,
    'exp': '',
    'is_train': True,
    'job': 8,
    'linear_size': 1024,
    'load': None,
    'lr': 0.001,
    'lr_decay': 5000,
    'lr_gamma': 0.9,
    'max_norm': True,
    'noise': None,
    'num_stage': 2,
    'out': '/data/LiftPose3D/capture/out_across_animals',
    'out_dir': '/data/LiftPose3D/capture/out_across_animals',
    'predict': False,
    'procrustes': False,
    'resume': False,
    'test': False}
[main.py:311]:DEBUG:==========================================

[lift.py:31]:INF

Epoch 000 | LR  0.00100 | Loss Test  0.00000 | Loss Train  0.65157|: 100%|██████████| 832/832 [00:06<00:00, 125.74it/s]
Epoch 001 | LR  0.00100 | Loss Test  0.33139 | Loss Train  0.10289|: 100%|██████████| 832/832 [00:06<00:00, 131.82it/s]
Epoch 002 | LR  0.00100 | Loss Test  0.23848 | Loss Train  0.08343|: 100%|██████████| 832/832 [00:06<00:00, 130.37it/s]
Epoch 003 | LR  0.00100 | Loss Test  0.21595 | Loss Train  0.07680|: 100%|██████████| 832/832 [00:06<00:00, 129.41it/s]
Epoch 004 | LR  0.00100 | Loss Test  0.19983 | Loss Train  0.07058|: 100%|██████████| 832/832 [00:06<00:00, 130.93it/s]
Epoch 005 | LR  0.00100 | Loss Test  0.20059 | Loss Train  0.06717|: 100%|██████████| 832/832 [00:06<00:00, 129.52it/s]
Epoch 006 | LR  0.00090 | Loss Test  0.21028 | Loss Train  0.06421|: 100%|██████████| 832/832 [00:06<00:00, 128.23it/s]
Epoch 007 | LR  0.00090 | Loss Test  0.19494 | Loss Train  0.06200|: 100%|██████████| 832/832 [00:06<00:00, 132.12it/s]
Epoch 008 | LR  0.00090 | Loss Test  0.1

KeyboardInterrupt: 

In [ ]:
%matplotlib inline
from liftpose.plot import read_log_train, plot_log_train
epoch, lr, loss_train, loss_test, err_test = read_log_train(par['out_dir'])
plot_log_train(plt.gca(), loss_train, loss_test, epoch)

In [ ]:
from liftpose.main import test as lp3d_test
lp3d_test(par['out_dir'])

In [ ]:
from liftpose.postprocess import load_test_results
from liftpose.vision_3d import camera_to_world, procrustes

data = torch.load(os.path.join(par['out_dir'], "test_results.pth.tar"))
stat_2d, stat_3d = (
    torch.load(os.path.join(par['out_dir'], "stat_2d.pth.tar")),
    torch.load(os.path.join(par['out_dir'], "stat_3d.pth.tar")),
)

test_3d_gt, test_3d_pred, good_keypts = load_test_results(data, stat_2d, stat_3d)

s = test_3d_gt.shape
test_3d_gt = camera_to_world(test_3d_gt.reshape(s[0], -1), cams['R'][0], cams['tvec'][0]).reshape(s)
test_3d_pred = camera_to_world(test_3d_pred.reshape(s[0],-1), cams['R'][0], cams['tvec'][0]).reshape(s)

#change occluded points to nana
ind_nan_pred = np.logical_not(test_keypoints)
ind_nan_gt = np.logical_not(test_keypoints)
ind_nan = (ind_nan_gt*ind_nan_pred).reshape(-1, 20, 3)
test_3d_pred[ind_nan] = np.nan
test_3d_gt[ind_nan] = np.nan

#make nans to zeros for procrustes
test_3d_pred[np.isnan(test_3d_pred)]=0
test_3d_gt[np.isnan(test_3d_gt)]=0

#remove all-0 poses
ind_0_pred = (test_3d_pred.sum(2).sum(1)>0)
ind_0_gt = (test_3d_gt.sum(2).sum(1)>0)
ind_0 = ind_0_pred*ind_0_gt
test_3d_pred = test_3d_pred[ind_0]
test_3d_gt = test_3d_gt[ind_0]
test_fid = test_fid[ind_0]
test_2d_sh = test_2d_sh[ind_0]

#procrustes align
test_3d_pred = procrustes(test_3d_gt, test_3d_pred)

#put nans back
test_3d_gt[ind_nan[ind_0]]=np.nan
test_3d_pred[ind_nan[ind_0]]=np.nan

# Visualize Predictions

In [ ]:
#%matplotlib widget
%matplotlib inline
from liftpose.plot import plot_pose_3d
fig = plt.figure(figsize=plt.figaspect(1), dpi=100)
ax = fig.add_subplot(111, projection='3d')
i=5
plot_pose_3d(ax=ax, tar=test_3d_gt[i],
             pred=test_3d_pred[i],
            bones=par_data["vis"]["bones"], 
            limb_id=par_data["vis"]["limb_id"], 
            colors=par_data["vis"]["colors"], 
            normalize=False, show_pred_always=True)


In [ ]:
%matplotlib inline
from celluloid import Camera
from liftpose.plot import plot_pose_2d, plot_pose_3d

fig = plt.figure(figsize=(10,5), dpi=300)
ax = fig.add_subplot(121, projection='3d')
ax2 = fig.add_subplot(122)

cam_list = ['R', 'L', 'E', 'U', 'S', 'U2']
camera = Camera(fig)
for t in range(0, 10):
    frame_id = test_fid[t][0]
    if os.path.isfile(f'/mnt/NAS/SG/CAPTURE/images/rat7M_e0/sample0_{frame_id}_Camera{cam_list[0]}.png'):
    
        plot_pose_3d(ax=ax, tar=test_3d_gt[t], 
            pred=test_3d_pred[t], 
            bones=par_data["vis"]["bones"], 
            limb_id=par_data["vis"]["limb_id"], 
            colors=par_data["vis"]["colors"],
            normalize=False,
            legend=True)
    
        ax2.imshow(plt.imread(f'/mnt/NAS/SG/CAPTURE/images/rat7M_e0/sample0_{frame_id}_Camera{cam_list[0]}.png'))
        
        pt = test_2d_sh[t].reshape(-1,2)
        plot_pose_2d(ax2, 
            pt, 
            bones=par_data["vis"]["bones"], 
            limb_id=par_data["vis"]["limb_id"], 
            colors=par_data["vis"]["colors"],
            normalize=False
        )
        ax2.axis('off')
        camera.snap()
        
camera.animate().save('prediction.mp4')

Bone lengths

In [ ]:
from liftpose.vision_3d import calculate_bone_length
bl_animal_1 = calculate_bone_length(train_3d, par_data['vis']['bones'])
bl_animal_2 = calculate_bone_length(test_3d, par_data['vis']['bones'])
plt.plot(bl_animal_1[:,3],label='animal 1')
plt.plot(bl_animal_2[:,3],label='animal 2')
plt.xlim([0,5000])
#plt.ylim([40,120])
plt.xlabel('Frames')
plt.ylabel('Bone length (mm)')
#plt.savefig('neck_length.svg')
plt.legend()

# Errors

In [ ]:
%matplotlib inline
from liftpose.plot import violin_plot
import random, string
plt.figure(figsize=(10,5), dpi=100)
names = ['HeadF','HeadB','HeadL','SpineF','SpineM','SpineL','Offset1','Offset2','HipL','HipR',
         'ElbowL','ArmL','ShoulderL','ShoulderR','ElbowR','ArmR','KneeR','KneeL','ShinL','ShinR']
ax = plt.gca()
violin_plot(ax, test_3d_gt=test_3d_gt, test_3d_pred=test_3d_pred, test_keypoints=np.ones_like(test_3d_gt),
                joints_name=names, order=names)
plt.savefig('errors_norm.svg')